In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [3]:
import import_ipynb
#we are importing the notebook files to avoid code redundancy
#python can do almost anything, except run fast
import s_metaheuristics as smt

In [5]:
import tabu_search as ts

In [6]:
graph, random_solution, total_edge_length = dg.read_instance('data\data70.txt')
num_nodes = len(graph)

In [7]:
def tabu_local_search_combo(graph, solution, max_iterations, neighbor_gen_func, local_search_func, change_func ,tabu_tenure):
    current_solution = deepcopy(solution)
    current_value = dg.calculate_total_edge_length(graph, current_solution)
    best_solution = current_solution.copy()
    best_value = current_value
    tabu_list = []

    best_i = -1

    for ind in range(max_iterations):
        #current_solution, _, _ = deepcopy(local_search_func(graph, current_solution, current_value, int(max_iterations/20), change_func))
        current_solution, _, _ = deepcopy(local_search_func(graph, current_solution, current_value, int(max_iterations*0.1), change_func))

        neighbors = neighbor_gen_func(current_solution)
        neighbors.sort(key=lambda solution: dg.calculate_total_edge_length(graph, solution))
        #print(neighbors[0], calculate_total_edge_length(graph, neighbors[0]))

        
        next_solution = None
        for neighbor in neighbors:
            if neighbor not in tabu_list and dg.calculate_total_edge_length(graph, neighbor) <= current_value:
                next_solution = neighbor
                break


        if next_solution is None:
            next_solution = neighbors[0]


        tabu_list.append(next_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)


        current_solution = deepcopy(next_solution)
        current_value = dg.calculate_total_edge_length(graph, current_solution)

        # Update best solution
        if current_value < best_value:
            #print(current_value)
            best_i = ind
            #print('promena best_i', best_i)
            best_solution = deepcopy(current_solution)
            best_value = current_value

    return best_solution, best_value, best_i

In [8]:
#test
#solution, value, iter = tabu_local_search_combo(graph, random_solution, 500, ts.tabu_inverse, smt.local_search, smt.make_change_scramble ,5)
#print(solution, value, iter)

In [9]:
#let's itterate through the best combinations, instead of manually calling each combo and remembering each result
neighbor_generators = [ts.tabu_swap_neighbors, ts.tabu_inverse, ts.tabu_inverse_complete, ts.tabu_scramble, ts.tabu_scramble_continuous]
change_functions = [smt.make_change_swap, smt.make_change_scramble, smt.make_change_inverse]
search_functions = [smt.local_search, smt.simulated_annealing]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

for neighbor_gen in neighbor_generators:
    for search in search_functions:
        for change in change_functions:

            start = time.time()    
            sol, val, iter = tabu_local_search_combo(graph,
                                          random_solution,
                                          500,
                                          neighbor_gen,
                                          search,
                                          change,
                                          5)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(neighbor_gen.__name__, search.__name__, change.__name__, ' : ', sol, val, iter, '\n')

            sols.append(sols)
            values.append(val)
            times.append(duration)

            methods.append(str(neighbor_gen.__name__ + ', ' + search.__name__ + ', ' + change.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(val), 'Time': duration})

            if val == best:
                if duration <= times[best_i]:    
                    best = val
                    best_i = number_of_combinations

            if val < best:
                best = val
                best_i = number_of_combinations

            average[0] += val
            average[1] += duration

            number_of_combinations += 1    



tabu_swap_neighbors local_search make_change_swap  :  [7, 49, 8, 22, 25, 66, 4, 68, 20, 41, 44, 5, 55, 26, 34, 24, 11, 57, 35, 13, 59, 14, 3, 33, 51, 36, 39, 69, 18, 28, 1, 19, 30, 38, 15, 17, 65, 47, 67, 61, 21, 12, 46, 50, 43, 31, 32, 27, 42, 6, 56, 54, 58, 48, 2, 10, 63, 23, 53, 60, 16, 29, 45, 62, 52, 40, 64, 37, 9, 0] 24695 121 

tabu_swap_neighbors local_search make_change_scramble  :  [0, 9, 69, 7, 64, 65, 37, 47, 52, 62, 40, 16, 67, 53, 29, 60, 46, 21, 2, 17, 15, 6, 38, 58, 50, 19, 56, 32, 63, 12, 45, 27, 30, 43, 4, 14, 34, 3, 1, 28, 54, 35, 36, 39, 11, 51, 18, 31, 61, 10, 23, 41, 26, 25, 33, 68, 57, 24, 44, 48, 42, 66, 59, 13, 20, 55, 5, 22, 49, 8] 24711 314 

tabu_swap_neighbors local_search make_change_inverse  :  [0, 8, 49, 5, 22, 55, 69, 64, 26, 20, 13, 57, 52, 34, 3, 51, 11, 62, 14, 24, 42, 66, 59, 4, 36, 44, 33, 1, 54, 41, 35, 63, 28, 60, 30, 58, 19, 38, 50, 10, 15, 6, 2, 43, 39, 32, 18, 27, 23, 61, 56, 68, 31, 21, 48, 45, 12, 29, 16, 53, 17, 25, 40, 67, 46, 47, 37, 65, 

KeyboardInterrupt: 

In [11]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/tabu_local_combo.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/tabu_local_combo.csv'), index=False)

,Dim,Method,Value,Time
0,70,"tabu_swap_neighbors, local_search, make_change...",24695,62.20
1,70,"tabu_swap_neighbors, local_search, make_change...",24711,63.91
2,70,"tabu_swap_neighbors, local_search, make_change...",24770,62.78
3,70,"tabu_swap_neighbors, simulated_annealing, make...",24749,63.10
4,70,"tabu_swap_neighbors, simulated_annealing, make...",25664,64.49
5,70,"tabu_swap_neighbors, simulated_annealing, make...",25261,63.56
6,70,"tabu_inverse, local_search, make_change_swap",24554,68.47
7,70,"tabu_inverse, local_search, make_change_scramble",24733,71.48
8,70,"tabu_inverse, local_search, make_change_inverse",24667,72.09
9,70,"tabu_inverse, simulated_annealing, make_change...",24847,69.03


In [12]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: tabu_scramble, local_search, make_change_inverse 24492 1301.84


,Dim,Method,Value,Time
0,70,"tabu_scramble, local_search, make_change_inverse",24492,1301.84


In [13]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'taloc_combo'] = values[best_i]
    df.loc[row_to_update.index, 'taloc_combo_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'taloc_combo': values[best_i], 'taloc_combo_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [14]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [24863.14, 943.19]


,Dim,Value,Time
0,70,24863.14,943.19


In [15]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'taloc_combo'] = average[0]
    df.loc[row_to_update.index, 'taloc_combo_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'taloc_combo': average[0], 'taloc_combo_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)